In [3]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//aqi_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation


# Training utils
import utils_xgboost

# Optuna
import optuna

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# Modelos
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

# Cargamos los datos

In [4]:
datos = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates = ['FECHAHORA'])

# Variables de entrenamiento

In [5]:
estacion = 4

variables = ["ANHO", 'DIA', 'MES', 'HORA', 'MINUTO', 'MP1', 'MP2_5', 'MP10', 
             'TEMPERATURA', 'HUMEDAD', 'PRESION', 'TEMPERATURA_PRONOSTICO', 
             'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO', 'DIA_SEM', 'TRAFICO' , 'AQI_MP10', 'AQI_MP2_5']

dependent = ['AQI_MP2_5']

number_of_features = len(variables)

training_days = 7 
forecast_days = 1 
samples_per_day = 288
step = 288/4

# Creamos una variable que nos diga con cuantos meses de entrenamiento queremos contar para el X_train
train_months = relativedelta(months = 12)

input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test



# Procesamiento previo a entrenamiento

In [6]:
%%time

X_train, y_train, X_test, y_test = utils_xgboost.get_everything(datos, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)

CPU times: total: 17.7 s
Wall time: 17.8 s


In [7]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)


(1433, 2033)
(1433, 288)
(333, 2033)
(333, 288)


# Prueba antes de usar optuna

In [8]:
%%time

params = {'learning_rate' : 0.025,
          'n_estimators' : 250,
          'max_depth': 2,
          'min_child_weight' : 1,
          'gamma': 0.0,
          'subsample': 0.98,
          'colsample_bytree': 0.98,
          'scale_pos_weight': 0.8,
          'seed': 42,
          'verbosity' : 0}


xgb_model = xgb.XGBRegressor(** params)

trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

prediction = trained_xgb_model.predict(X_test)
print('prediction', prediction.shape)
print('test', y_test.shape)

MSE = np.mean((prediction - y_test)**2)
print('RMSE: ', MSE**0.5)
MAE = np.mean(np.abs(prediction-y_test))
print('MAE: ', MAE)

prediction (333, 288)
test (333, 288)
RMSE:  9.781115562652332
MAE:  6.694983114898304
CPU times: total: 1h 56min 35s
Wall time: 10min 16s


In [8]:
%%time

def objective(trial):
    
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 10, 300),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
    }
    
    optuna_xgb_model = xgb.XGBRegressor(** params)
    
    trained_xgb_model = MultiOutputRegressor(optuna_xgb_model).fit(X_train , y_train)

    prediction = trained_xgb_model.predict(X_test)

    MSE = np.mean((prediction - y_test)**2)
    RMSE = MSE**0.5
    print('RMSE: ', RMSE)
    MAE = np.mean(np.abs(prediction - y_test))
    print('MAE: ', MAE)

    MAPE = mean_absolute_percentage_error(prediction, y_test)
    
    return MAPE

study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials = 100)

trial = study.best_trial

[I 2023-02-19 17:23:11,368] A new study created in memory with name: no-name-85115f8f-4ed0-4657-8f73-17b438165ed5
[I 2023-02-19 17:25:54,828] Trial 0 finished with value: 0.34230081238686033 and parameters: {'max_depth': 2, 'learning_rate': 0.4667027619890214, 'n_estimators': 65, 'min_child_weight': 3, 'gamma': 0.0025513651991129526, 'subsample': 0.7112199843708198, 'colsample_bytree': 0.7601960490661962}. Best is trial 0 with value: 0.34230081238686033.


RMSE:  10.565794024252202
MAE:  7.303961083456319


[I 2023-02-19 17:32:16,152] Trial 1 finished with value: 2.427105193363474 and parameters: {'max_depth': 5, 'learning_rate': 0.5248169860666795, 'n_estimators': 297, 'min_child_weight': 7, 'gamma': 0.002695356670957413, 'subsample': 0.11648149049499729, 'colsample_bytree': 0.024453640315587572}. Best is trial 0 with value: 0.34230081238686033.


RMSE:  170.95876243071902
MAE:  125.60573406030208


[I 2023-02-19 17:33:42,154] Trial 2 finished with value: 0.29091129700188245 and parameters: {'max_depth': 8, 'learning_rate': 0.08604309342796732, 'n_estimators': 47, 'min_child_weight': 10, 'gamma': 2.7095902920538166e-05, 'subsample': 0.2425498678354642, 'colsample_bytree': 0.0783788463871129}. Best is trial 2 with value: 0.29091129700188245.


...

RMSE:  9.742903613138033
MAE:  6.591258484058512


[I 2023-02-20 18:06:31,659] Trial 98 finished with value: 0.2661199216273335 and parameters: {'max_depth': 6, 'learning_rate': 0.014704734548187864, 'n_estimators': 274, 'min_child_weight': 9, 'gamma': 0.012117280349612087, 'subsample': 0.31772034177577196, 'colsample_bytree': 0.3892324290321974}. Best is trial 92 with value: 0.2635610550252555.


RMSE:  9.767764487211041
MAE:  6.528148721984358


[I 2023-02-20 18:21:43,168] Trial 99 finished with value: 0.2651530161383691 and parameters: {'max_depth': 5, 'learning_rate': 0.012053332670332297, 'n_estimators': 282, 'min_child_weight': 10, 'gamma': 0.002924028352944537, 'subsample': 0.2811131574108288, 'colsample_bytree': 0.5763151263051624}. Best is trial 92 with value: 0.2635610550252555.


RMSE:  9.54610848417156
MAE:  6.420640103799882
CPU times: total: 11d 5h 34min 52s
Wall time: 1d 58min 31s


In [1]:
import joblib

joblib.dump(study, "optuna_studies/study_XGBOOST_230219_e4_7antes_1prediccion_cada6h_MAPE.pkl")
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

NameError: name 'study' is not defined